# Pandas: scaling to large datasets

In [1]:
import random
import string
import numpy as np
import pandas as pd
from datetime import datetime
import pathlib
%load_ext memory_profiler

Create a large dataset

In [2]:
%%time
def gen_random_string(length:int=32) -> str:
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=length))
    
def make_timeseries(start="2000-01-01", end="2000-12-31", freq="1D", seed=None):

    index = pd.date_range(start=start, end=end, freq=freq, name="timestamp")
    n = len(index)
    np.random.seed = seed
    columns = {
        'cat': np.random.choice(['cat1','cat2','cat3','cat4','cat5'],n),
        'str1':[gen_random_string() for _ in range(n)],
        'str2':[gen_random_string() for _ in range(n)],
        'a': np.random.rand(n),
        'b': np.random.rand(n),
        'c': np.random.randint(1,100,n),
    }

    df = pd.DataFrame(columns, index=index, columns=sorted(columns))
    if df.index[-1] == end:
        df = df.iloc[:-1]
    return df

timeseries = [
    make_timeseries(start=datetime(2020,1,1), end=datetime(2023,12,31), freq='1min', seed=10).rename(columns=lambda x: f"{x}_{i}")
    for i in range(5)
]
df = pd.concat(timeseries, axis=1)

CPU times: user 1min 27s, sys: 1.61 s, total: 1min 28s
Wall time: 1min 28s


Print the fisrt rows to see what the data looks like.

In [3]:
df.head()

,a_0,b_0,c_0,cat_0,str1_0,str2_0,a_1,b_1,c_1,cat_1,...,c_3,cat_3,str1_3,str2_3,a_4,b_4,c_4,cat_4,str1_4,str2_4
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-01-01 00:00:00,0.439790,0.069992,85,cat1,25CH60DTU73CZ67GR54GS83RD6N3ARPP,8P63JISH1ZP47MIAZBI12V1JXGK58IX8,0.265880,0.903861,82,cat2,...,13,cat4,NBVW8J14DZTVEDHISETW5GAK7EF17GXG,OEZF17L5NT64ZJB0KDDPXLGZ65OZAHSO,0.687856,0.678377,34,cat4,TEJ5L414VFDEE80ZKN4MOTT55W84VIV1,04ZMT2BQ1A9LDVL5C8FAMYIH5UWBU0UI
2020-01-01 00:01:00,0.307949,0.737754,28,cat4,WU6SF4XH0N297NGVQRVT4HJPRHFYY3K0,DHR0FWQD06X1373HHDCZIH7XVYN4ATNP,0.431852,0.119335,21,cat2,...,73,cat2,HZRH5W38UBQR9OV6BV3FY8EMTP33HSPB,HNUFOM1BLH8E30XVTFSFK13AZP23KX2Z,0.186750,0.882835,43,cat5,LAEIZ0WESDJ6JZ2E0M8LMQAH7ZBZEHY8,GQT42GP69K4YZ50H00I695KRSEBX87BB
2020-01-01 00:02:00,0.844226,0.251170,31,cat2,CXHNSBLSIIPQ9V5CBM1XA13APFKC4TX2,46YDT9RO6UUM9ZEVE537U2WH9M9YANX2,0.170822,0.954205,11,cat5,...,6,cat2,NI4WLNG33HG36JSWYSPJR1A411L86UYM,P6BULVBOV581SOARU9QTSQK8WW4HRKKS,0.864973,0.355677,71,cat5,3596BQIYQQ0FLWQDBRM9NFTRGJVOLPVH,AW7H0ELFQFU75NVEN5ZU5AK1FGV0LAG4
2020-01-01 00:03:00,0.509127,0.226031,22,cat2,24PBVSV29Z015YFR8XR0FM88YBXWGP52,X24BJX4X4H0SXXSTER0WY2OG8H51OPRP,0.437260,0.907608,42,cat4,...,5,cat4,JN91A6OZKPZVOQ659KMWBCI0S314QDP3,JXZPV75E16IKN066HF1YEZN818R3DBA2,0.945028,0.491691,87,cat2,HF6EQ8TJ923PM3028ZER7812CV2M2HN4,AUGEW9I2R1L7EBJ1TJBK5V765S24SBKZ
2020-01-01 00:04:00,0.845486,0.419131,78,cat5,PF8D3NWV0EP941UNREM9JO094PQNJDR9,HPO5JJA877ORFQXB6NWIG2MCRF2G38YP,0.385516,0.230385,12,cat4,...,72,cat4,ZSX13ZKIUQ1JB9V5QTM05D7O8SQJFOIN,9MVSK7KZ3FPJZYVKMOKQ9OF2UYZY2VJ6,0.328728,0.631368,3,cat2,2VMO6KZ0NR5YYBMUUJ4LZ2GUGFT1PQSX,GGOND089HZEJEXNECS4RMFP5PFA7JNT6


The method `info(memory_usage='deep')` returns the column types and also gives the memory usage of the dataframe.

In [4]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2102400 entries, 2020-01-01 00:00:00 to 2023-12-30 23:59:00
Freq: min
Data columns (total 30 columns):
 #   Column  Dtype  
---  ------  -----  
 0   a_0     float64
 1   b_0     float64
 2   c_0     int64  
 3   cat_0   object 
 4   str1_0  object 
 5   str2_0  object 
 6   a_1     float64
 7   b_1     float64
 8   c_1     int64  
 9   cat_1   object 
 10  str1_1  object 
 11  str2_1  object 
 12  a_2     float64
 13  b_2     float64
 14  c_2     int64  
 15  cat_2   object 
 16  str1_2  object 
 17  str2_2  object 
 18  a_3     float64
 19  b_3     float64
 20  c_3     int64  
 21  cat_3   object 
 22  str1_3  object 
 23  str2_3  object 
 24  a_4     float64
 25  b_4     float64
 26  c_4     int64  
 27  cat_4   object 
 28  str1_4  object 
 29  str2_4  object 
dtypes: float64(10), int64(5), object(15)
memory usage: 2.6 GB


Write the dataframe 

In [5]:
pathlib.Path("data").mkdir(parents=True,exist_ok=True)
df.to_parquet("timeseries.parquet")

## Load only useful data

Image that you are interested only by 

Imagine you're only interested in a subset of the dataset's columns `['a_0','a_1','cat_0','str1_0','str1_1']`. Then there are two ways to proceed: 
 * either load the entire dataset and then filter out the columns you're interested in
 * or read only the columns you're interested in

Compare the two loading methods.

Look at the `read_parquet`method

In [6]:
?pd.read_parquet

Signature:
pd.read_parquet(
    path: 'FilePath | ReadBuffer[bytes]',
    engine: 'str' = 'auto',
    columns: 'list[str] | None' = None,
    storage_options: 'StorageOptions | None' = None,
    use_nullable_dtypes: 'bool | lib.NoDefault' = <no_default>,
    dtype_backend: 'DtypeBackend | lib.NoDefault' = <no_default>,
    filesystem: 'Any' = None,
    filters: 'list[tuple] | list[list[tuple]] | None' = None,
    **kwargs,
) -> 'DataFrame'
Docstring:
Load a parquet object from the file path, returning a DataFrame.

Parameters
----------
path : str, path object or file-like object
    String, path object (implementing ``os.PathLike[str]``), or file-like
    object implementing a binary ``read()`` function.
    The string could be a URL. Valid URL schemes include http, ftp, s3,
    gs, and file. For file URLs, a host is expected. A local file could be:
    ``file://localhost/path/to/table.parquet``.
    A file URL can also be a path to a directory that contains multiple
    partitioned p

In [7]:
columns = ['a_0','a_1','cat_0','str1_0','str1_1']

**Option 1**: Load the entire dataset and then filter out the columns you're interested in

In [8]:
# TODO

In [9]:
%memit df_filter = pd.read_parquet("timeseries.parquet")[columns]

peak memory: 8360.64 MiB, increment: 3850.33 MiB


In [10]:
%%time
df_filter = pd.read_parquet("timeseries.parquet")[columns]
df_filter.head()

CPU times: user 7.39 s, sys: 5.32 s, total: 12.7 s
Wall time: 7.8 s


,a_0,a_1,cat_0,str1_0,str1_1
timestamp,,,,,
2020-01-01 00:00:00,0.439790,0.265880,cat1,25CH60DTU73CZ67GR54GS83RD6N3ARPP,I2IPGR00OEV176DVCR80BJQGMILCL27U
2020-01-01 00:01:00,0.307949,0.431852,cat4,WU6SF4XH0N297NGVQRVT4HJPRHFYY3K0,SIO7HVCE7ST8D79QPC0UKBYSBJTFTG6K
2020-01-01 00:02:00,0.844226,0.170822,cat2,CXHNSBLSIIPQ9V5CBM1XA13APFKC4TX2,OXRZBU95BT844XTKK9GX0DQ7GJNFJZW9
2020-01-01 00:03:00,0.509127,0.437260,cat2,24PBVSV29Z015YFR8XR0FM88YBXWGP52,VNURGQNX1O89LBFOK0TSJKYK5MQ1E2G3
2020-01-01 00:04:00,0.845486,0.385516,cat5,PF8D3NWV0EP941UNREM9JO094PQNJDR9,GPC1DA2B8CS1J9APR76ATO0GJKUFIAK4


**Option 2**: Read only the columns you're interested in. 

In [11]:
# TODO

In [12]:
%memit df_filter = pd.read_parquet("timeseries.parquet",columns=columns)

peak memory: 6119.67 MiB, increment: -334.02 MiB


In [13]:
%%time
df_filter = pd.read_parquet("timeseries.parquet",columns=columns)
df_filter.head()

CPU times: user 1.4 s, sys: 1.18 s, total: 2.57 s
Wall time: 1.83 s


,a_0,a_1,cat_0,str1_0,str1_1
timestamp,,,,,
2020-01-01 00:00:00,0.439790,0.265880,cat1,25CH60DTU73CZ67GR54GS83RD6N3ARPP,I2IPGR00OEV176DVCR80BJQGMILCL27U
2020-01-01 00:01:00,0.307949,0.431852,cat4,WU6SF4XH0N297NGVQRVT4HJPRHFYY3K0,SIO7HVCE7ST8D79QPC0UKBYSBJTFTG6K
2020-01-01 00:02:00,0.844226,0.170822,cat2,CXHNSBLSIIPQ9V5CBM1XA13APFKC4TX2,OXRZBU95BT844XTKK9GX0DQ7GJNFJZW9
2020-01-01 00:03:00,0.509127,0.437260,cat2,24PBVSV29Z015YFR8XR0FM88YBXWGP52,VNURGQNX1O89LBFOK0TSJKYK5MQ1E2G3
2020-01-01 00:04:00,0.845486,0.385516,cat5,PF8D3NWV0EP941UNREM9JO094PQNJDR9,GPC1DA2B8CS1J9APR76ATO0GJKUFIAK4


You can use the magic command `%time` and `%memit` to compare the time and the memory usage of the two calls.

Not all the reading methods in Pandas has an option to read a subset of columns.

### Use efficient datatypes

The default pandas data types are not the most memory efficient. This is especially true for text data columns with relatively few unique values (commonly referred to as “low-cardinality” data). 

Using more efficient data types reduces the memory size of a dataframe, so you can store larger datasets in memory.

In [14]:
df = pd.read_parquet("timeseries.parquet",columns=['a_0','b_0','c_0','cat_0','str1_0','str2_0'])

Look at the data types of each column

In [15]:
df.dtypes

a_0       float64
b_0       float64
c_0         int64
cat_0      object
str1_0     object
str2_0     object
dtype: object

Look at the memory usage of the dataframe. The `memory_usage()` method returns the memory usage of each column in bytes.

In [16]:
df.memory_usage(deep=True)

Index      16819200
a_0        16819200
b_0        16819200
c_0        16819200
cat_0     128246400
str1_0    187113600
str2_0    187113600
dtype: int64

Compute the size of the dataframe. You should get the same result with the `info(memory_usage='deep')` method.

In [17]:
# TODO

In [18]:
mem = df.memory_usage(deep=True)
mem.sum()/1024/1024

543.3563232421875

In [19]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2102400 entries, 2020-01-01 00:00:00 to 2023-12-30 23:59:00
Data columns (total 6 columns):
 #   Column  Dtype  
---  ------  -----  
 0   a_0     float64
 1   b_0     float64
 2   c_0     int64  
 3   cat_0   object 
 4   str1_0  object 
 5   str2_0  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 543.4 MB


The result of `memory_usage` show that the columns taking up much more memory are 'str1_0','str2_0','cat_0'. It seems normal for 'str1_0','str2_0' columns because those columns contains random strings. But 'cat_0' column has just a few unique values, so it’s a good candidate for converting to a pandas.Categorical. With a pandas.Categorical, we store each unique name once and use space-efficient integers to know which specific name is used in each row.

First, we copy our dataframe to a new one.

In [20]:
df2 = df.copy()

Try to change to column type to Pandas.category using the `astype()` method

In [21]:
# TODO

In [22]:
df2["cat_0"] = df2["cat_0"].astype("category")

Check with dtypes that the column type has changed

In [23]:
# TODO

In [24]:
df2.dtypes

a_0        float64
b_0        float64
c_0          int64
cat_0     category
str1_0      object
str2_0      object
dtype: object

Compute the memory usage of each column for this new dataframe.

In [25]:
# TODO

In [26]:
df2.memory_usage(deep=True)

Index      16819200
a_0        16819200
b_0        16819200
c_0        16819200
cat_0       2102877
str1_0    187113600
str2_0    187113600
dtype: int64

We can go a bit further and downcast the numeric columns to their smallest types using pandas.to_numeric(). The "c_0" column contains number between 0 and 100. So it can be downcast to unsigned. If float precision is sufficient for columns 'a_0' et 'b_0', it is also possible to downcast to float. Be careful when you downcast, you lose precision and so you can propagate error during the processing.

In [27]:
# TODO

In [28]:
df2["c_0"] = pd.to_numeric(df2["c_0"], downcast="unsigned")
df2[["a_0", "b_0"]] = df2[["a_0", "b_0"]].apply(pd.to_numeric, downcast="float")

Check the types and the memory usage of the columns

In [29]:
# TODO

In [30]:
df2.dtypes

a_0        float32
b_0        float32
c_0          uint8
cat_0     category
str1_0      object
str2_0      object
dtype: object

In [31]:
df2.memory_usage(deep=True)

Index      16819200
a_0         8409600
b_0         8409600
c_0         2102400
cat_0       2102877
str1_0    187113600
str2_0    187113600
dtype: int64

Compute the memory reduction

In [32]:
# TODO

In [33]:
reduction = df2.memory_usage(deep=True).sum() / df.memory_usage(deep=True).sum()
print(f"{reduction:0.2f}")

0.72


# Use chunking

Some problem are embarrasingly parallel and so can be processed with chunking, which means by splitting a large problem into a bunch of small problems. 
For example, converting an big file into several smaller files and repeating the processing for each file in a directory. 
As long as each chunk fits in memory, you can work with datasets that are much larger than memory.

In [34]:
N = 12
starts = [f"20{i:>02d}-01-01" for i in range(N)]
ends = [f"20{i:>02d}-12-31" for i in range(N)]
pathlib.Path("data/timeseries").mkdir(parents=True,exist_ok=True)
for i, (start, end) in enumerate(zip(starts, ends)):
    ts = make_timeseries(start=start, end=end, freq="1min", seed=i)
    ts.to_parquet(f"data/timeseries/ts-{i:0>2d}.parquet")

Count the occurence of the values in the "c" column for all the files.

In [35]:
# TODO

In [36]:
%%time

files = pathlib.Path("data/timeseries/").glob("ts*.parquet")
counts = pd.Series(dtype=int)

for path in files:
    df = pd.read_parquet(path)
    counts = counts.add(df["c"].value_counts(), fill_value=0)

counts.astype(int)

CPU times: user 3.66 s, sys: 2.32 s, total: 5.98 s
Wall time: 4.5 s


c
1     63221
2     63234
3     63501
4     64154
5     63608
      ...  
95    63308
96    63758
97    63770
98    63447
99    63236
Length: 99, dtype: int64

Some readers, like pandas.read_csv(), offer parameters to control the chunksize when reading a single file. 
In that case, it is possible to read a file chunk by chunk in order to process it.

In [37]:
df = make_timeseries(start="2023-01-01", end="2023-12-31", freq="1min", seed=10)
df.to_csv("data/timeseries.csv")

Try to count the occurence of the values in the "c" column for the CSV file by process it chunk by chunk. You need to use the parameter `chunksize` in the `read_csv`method. 

In [38]:
# TODO

In [39]:
counts = pd.Series(dtype=int)
with pd.read_csv("data/timeseries.csv",chunksize=1000) as reader:
    for chunk in reader:
        counts = counts.add(chunk["c"].value_counts(), fill_value=0)

counts.astype(int)

c
1     5338
2     5330
3     5334
4     5272
5     5282
      ... 
95    5197
96    5215
97    5344
98    5296
99    5212
Length: 99, dtype: int64

In [40]:
%%memit
counts = pd.Series(dtype=int)
with pd.read_csv("data/timeseries.csv",chunksize=1000) as reader:
    for chunk in reader:
        counts = counts.add(chunk["c"].value_counts(), fill_value=0)

counts.astype(int)

peak memory: 6178.48 MiB, increment: 0.00 MiB


In [43]:
%%memit
df = pd.read_csv("data/timeseries.csv")
df["c"].value_counts().astype(int)

peak memory: 6654.18 MiB, increment: 113.72 MiB
